# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합-모듈화**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * Python 코드 모듈화
        * 각 모듈 코드 및 모델, 데이터파일을 일관성 있게 정리
        * .py 파일 생성 ==> 라이브러리 로딩, 각 task를 위한 함수 생성


## **1.환경설정**

* 경로 설정

구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/project6_2/'

## 2.모듈 구성하기

In [ ]:
%%writefile /content/drive/MyDrive/project6_2/emergency.py

import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch


# 0. load key file------------------
def load_api_keys(file_path):
    """
    API 키 파일에서 키를 로드합니다.

    Parameters:
        file_path (str): 키 파일 경로.

    Returns:
        dict: API 키 딕셔너리.
    """
    try:
        with open(file_path, 'r') as file:
            keys = json.load(file)
        return keys
    except FileNotFoundError:
        raise FileNotFoundError(f"키 파일 어디...?")
    except json.JSONDecodeError:
        raise ValueError("API 키 파일 형식 무엇...?")


# 1-1 audio2text--------------------


# 1-2 text2summary------------------


# 2. model prediction------------------


# 3-1. get_distance-----------------
class EmergencyRoomLocator:
    """
    네이버 지도 API를 사용하여 가장 가까운 응급실을 검색하는 클래스입니다!
    """

    def __init__(self, client_id, client_secret, csv_path, initial_radius=10, radius_increment=10, unit='km'):
        """
        클래스 초기화와 로드부분

        Parameters:
            client_id (str): 클라이언트 아이디
            client_secret (str): 클라이언트 비밀번호
            csv_path (str): 응급실 csv경로
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.initial_radius = initial_radius
        self.radius_increment = radius_increment
        self.unit = unit

        # 응급실 데이터 로드
        self.emergency_data = pd.read_csv(csv_path)

    def get_distance_from_naver_api(self, start_lat, start_lng, dest_lat, dest_lng):
        """
        네이버 지도 API를 사용하여 두 좌표 간의 도로 거리(km)를 계산합니다.

        Parameters:
            start_lat (float): 출발지의 위도
            start_lng (float): 출발지의 경도
            dest_lat (float): 목적지의 위도
            dest_lng (float): 목적지의 경도

        Returns:
            float: 도로 거리
        """
        url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.client_id,
            "X-NCP-APIGW-API-KEY": self.client_secret,
        }
        params = {
            "start": f"{start_lng},{start_lat}",
            "goal": f"{dest_lng},{dest_lat}",
            "option": "trafast"
        }
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            try:
                distance = data['route']['trafast'][0]['summary']['distance'] / 1000
                return round(distance, 2)
            except KeyError:
                return float('inf')
        else:
            raise ConnectionError(f"API 호출 실패: {response.status_code}, {response.text}")

# 3-2. recommendation------------------

    def find_nearest_emergency_rooms(self, input_coords, top_n=3):
        """
        입력된 좌표를 기준으로 도로 거리 기반 가장 가까운 응급실을 검색합니다.

        Parameters:
            input_coords (tuple): 입력 좌표 (위도, / 경도).
            top_n (int): 반환할 응급실 개수.

        Returns:
            pd.DataFrame: 가장 가까운 응급실 정보와 거리.
        """
        radius = self.initial_radius
        lat, lon = input_coords

        while True:
            R_km = 6371
            delta_lat = radius / R_km
            delta_lon = radius / (R_km * cos(radians(lat)))

            filtered_data = self.emergency_data[
                (self.emergency_data['위도'] >= lat - delta_lat) &
                (self.emergency_data['위도'] <= lat + delta_lat) &
                (self.emergency_data['경도'] >= lon - delta_lon) &
                (self.emergency_data['경도'] <= lon + delta_lon)
            ].copy()

            if not filtered_data.empty:
                filtered_data['Distance'] = filtered_data.apply(
                    lambda row: self.get_distance_from_naver_api(
                        lat, lon, row['위도'], row['경도']
                    ),
                    axis=1
                )
                filtered_data = filtered_data[filtered_data['Distance'] < float('inf')]
                nearest_emergency_rooms = filtered_data.sort_values(by='Distance').head(top_n)
                if len(nearest_emergency_rooms) >= top_n:
                    return nearest_emergency_rooms

            radius += self.radius_increment